In [ ]:
from pathlib import Path
from time import time
import pandas as pd
from videosentimentanalysis.usecases.download_video_use_case import VideoUseCase
from videosentimentanalysis.adapters.logging.custom_logger import CustomLogger
from videosentimentanalysis.adapters.logging.loguru_logger import LoguruLogger
from videosentimentanalysis.adapters.extract_audio.moviepy_extraction import MoviePyAudioExtractor
from videosentimentanalysis.usecases.video_utility_use_case import VideoUtilityUseCase
from videosentimentanalysis.adapters.transcribe_audio.speechrecognition import SpeechRecognition
from videosentimentanalysis.adapters.perform_sentiment_analysis.textblob_analysis import TextBlobAnalysis
from videosentimentanalysis.adapters.translate_text.google_translate import TextTranslation
from videosentimentanalysis.usecases.protocols.translate_text import LanguageOptions
from videosentimentanalysis.adapters.extract_emotions.text2emotion_extractor import GetEmotions
from videosentimentanalysis.domain.audio import Audio
import json
from threading import Thread
from multiprocessing import Process, Manager


## Constants

In [ ]:
OUTPUT_FOLDER= Path("./output/")


To run this project you need to have the following installed:
- ffmpeg
- python3.12
- poetry

## Setup Adapters
### Logging


In [ ]:
custom_logger = CustomLogger(Path("./download_log.txt"))
loguru_logger = LoguruLogger(Path("./loguru_logger.log"))

### Video Utilities

In [ ]:

moviepy_extractor = MoviePyAudioExtractor(Path("./tmp_audio"))
speechrecognition = SpeechRecognition()
sentimentanalysis = TextBlobAnalysis()
text_translation = TextTranslation()
emotion_extraction = GetEmotions()

Load all the Youtube URLs to a Pydantic Model called RawURLVideo

In [ ]:
video_use_case = VideoUseCase(video_output_directory=Path("tmp_videos"), logger=custom_logger)

In [ ]:
raw_url_videos=video_use_case.load_raw_url_video(Path('./video_urls.txt'))
raw_url_videos

In [ ]:
sequential_time_taken = time()
video_use_case.download_raw_videos_sequentially(raw_url_videos)
sequential_time_taken = time() - sequential_time_taken

In [ ]:
threading_time_taken = time()
video_use_case.download_raw_videos_threading(raw_url_videos)
threading_time_taken = time() - threading_time_taken

In [ ]:
process_time_taken = time()
videos=video_use_case.download_raw_videos_multiprocessing(raw_url_videos)
process_time_taken = time() - process_time_taken

## Results of each method for downloading videos and time taken

In [ ]:
pd.DataFrame({
    "Method": ["Sequential", "Threading", "Multiprocessing"],
    "Time Taken": [sequential_time_taken, threading_time_taken, process_time_taken]
})

The above shows the difference in performance when used downloading videos sequentially, using threading or multiprocessing. Unsurprisingly, the sequential method is the slowest. As this is a CPU bound task, the multiprocessing method is the quickest one.

In [ ]:

for video_index in range(len(videos)):
    video=videos[video_index]
    video_utlity_use_case=VideoUtilityUseCase(video=video, logger=custom_logger, extract_audio=moviepy_extractor, extract_text=speechrecognition, extract_polarity_and_sensitivity=sentimentanalysis, translate=text_translation,extract_emotions=emotion_extraction)
    
    video_output_folder= OUTPUT_FOLDER / video.title
    video_output_folder.mkdir(exist_ok=True, parents=True)
    
    video.move(video_output_folder)
    print(video.local_storage_path)
    
    audio = video_utlity_use_case.extract_audio()
    audio.move(video_output_folder)
    
    with open(video_output_folder / "sentimental_analysis.json", mode="w") as sentimental_analysis_file:
        sentimental_analysis_file.write(json.dumps(video_utlity_use_case.get_sentiment_analysis()))
    
    with open(video_output_folder / "transcribe_text.txt", mode="w") as transcribe_text_file:
        transcribe_text_file.write(video_utlity_use_case.transcribe_audio())
    
    with open(video_output_folder / "translate_text.txt", mode="w") as translate_text_file:
        translate_text_file.write(video_utlity_use_case.translate_text(source_lang=LanguageOptions.ENGLISH, target_lang=LanguageOptions.SPANISH))
    
    with open(video_output_folder / "detect_emotions.json", mode="w") as detect_emotions_file:
        detect_emotions_file.write(json.dumps(video_utlity_use_case.detect_emotions()))
    


### Sequential time for extract audio

In [ ]:
videos=video_use_case.download_raw_videos_multiprocessing(raw_url_videos)

In [ ]:
extract_audio_time_seq = time()
for video in videos:
    video_utlity_use_case=VideoUtilityUseCase(video=video, logger=custom_logger, extract_audio=moviepy_extractor, extract_text=speechrecognition, extract_polarity_and_sensitivity=sentimentanalysis, translate=text_translation,extract_emotions=emotion_extraction)
    video_utlity_use_case.extract_audio()

extract_audio_time_seq = time() -extract_audio_time_seq
print(f"Time taken for sequential extraction is {extract_audio_time_seq} seconds")

### Threading time for extract audio

In [ ]:
def threading_wrapper_extractor(results: list[Audio],index: int,video_utlity_use_case :  VideoUtilityUseCase):
    custom_logger.log(f"Processing {video_utlity_use_case.video.title} on thread {index}")
    results[index]=video_utlity_use_case.extract_audio()

In [ ]:
extract_audio_time_threading = time()
results = [None] * len(videos)
threads = []
for index, video in enumerate(videos):
    video_utlity_use_case= VideoUtilityUseCase(video=video, logger=custom_logger, extract_audio=moviepy_extractor, extract_text=speechrecognition, extract_polarity_and_sensitivity=sentimentanalysis, translate=text_translation,extract_emotions=emotion_extraction)
    thread = Thread(target=threading_wrapper_extractor, args=(results,index,video_utlity_use_case))
    threads.append(thread)
    thread.start()
for thread in threads:
    thread.join()
extract_audio_time_threading = time() -extract_audio_time_threading
print(f"Time taken for threading extraction is {extract_audio_time_threading} seconds")

### Processing time for extract audio

In [ ]:
extract_audio_time_processes = time()
manager = Manager()
results = manager.list([None] * len(videos))
processes = []
for index, video in enumerate(videos):
    video_utlity_use_case= VideoUtilityUseCase(video=video, logger=custom_logger, extract_audio=moviepy_extractor, extract_text=speechrecognition, extract_polarity_and_sensitivity=sentimentanalysis, translate=text_translation,extract_emotions=emotion_extraction)
    process = Process(target=video_utlity_use_case._multiprocessing_wrapper_audio_extraction_wrapper, args=(results,index))
    processes.append(process)
    process.start()
for process in processes:
    process.join()
extract_audio_time_processes = time() -extract_audio_time_processes
print(f"Time taken for processing extraction is {extract_audio_time_processes} seconds")

### Comparison of sequential, threading, multiprocessing for extracting audio

In [ ]:
pd.DataFrame({
    "Method": ["Sequential", "Threading", "Multiprocessing"],
    "Time Taken": [extract_audio_time_seq, extract_audio_time_threading, extract_audio_time_processes]
})

## Short Report

# CineSense Project Implementation:


## Architecture

I’ve selected the hexagonal (also called ports and adapters) architecture for this project as it is widely used in industry settings due to its scalability and reusability. It has been used by Netflix to manage their flow of large amounts of data from different sources due to its ability to  swap data sources without impacting business logic (Damir Svrtan 2020). 
The architecture breaks the program down into Domain, Services, Use cases and Adapters. The Domain is the broad data that we’re working with – in this case YouTube videos and their content. Services represent the results we want to obtain from the domain – in this case a Jupyter file which presents the log of what we did with the data, and the #1 downloaded videos, #2 transcribed audio, #3 sentiment analysis, #4 emotion extraction and #5 text translation into Spanish – the result of which can be found in the output files. Because the actual implementation – which lives in the adapters folder – is decoupled from the business logic, the application can be easily maintained should the requirements change – e.g. if we want to use a different library to extract emotions, we simply modify the adapter. 

## Use cases

There are two major use cases: the actual downloading of the YouTube videos and the actions we can perform on the content, such as sentiment analysis. I have enveloped the actions into a single use case called utilities to separate the obtaining of the data and the analysis of the data. 
Downloading Videos
Downloading videos only requires one method. I have compared the speed of download for sequential download, threading and multiprocessing. The results of this can be found in the pandas data frame in the Jupyter file. Unsurprisingly, the sequential method is the least efficient. The multiprocessing method is the most efficient, as this is a CPU bound task and multiprocessing is taking advantage of multi-core processors and processes downloads in parallel. The time saving is evident, as multiprocessing is roughly 4x quicker than sequential download in this case. Threading also brings a much improved result compared to sequential download, however, as this is not an I/O bound task, it is less efficient than multiprocessing. 

## Analysing Videos
The video analysis consists of extracting audio, transcribing audio, extracting sentiment analysis and extracting emotions. I have used the MoviePy library for extracting the audio and speechrecognition for transcribing it. For extracting emotions I’ve used the text2emotion library, for sentiment analysis I’ve used the TextBlob library (returning the polarity and sensitivity) and for translation I’ve relied on Google Translate. The results for each video can be found in the respective folder, labelled according to the name of the video. 
As the extract emotions, extract sentiment analysis and translate methods repeatedly rely on the results of the extract audio and transcribe audio methods, I have used caching to store the extracted and transcribed audio, which significantly increased the overall speed of the program. 
I have also compared the sequential, threading and multiprocessing methods for extracting audio. Once again, the multiprocessing method is the most efficient due to its suitability for CPU bound tasks. Surprisingly, the sequential method performed marginally better than the threading method. There is a variety of possible reasons for this, with a likely one being that threading creates a lot of overhead in terms of managing context switching and synchronization. Furthermore, threads working on extracting one audio might lead to contention for resources, which is avoided when using the sequential method. 

## Logging

I have implemented a simple logger to inform the user regarding the state of the processing of individual videos, as well as any potential errors. 
There is also the option to use loguru for a more scaled up logging experience. 


## Works Cited
Damir Svrtan, Sergii Makagon. 2020. Netflix Technology Blog. 10 March. Accessed June 24, 2024. https://netflixtechblog.com/ready-for-changes-with-hexagonal-architecture-b315ec967749.


